In [1]:
#I have taken exmaple of fake news; the dataset has been taken from Kaggle


In [2]:
#Installing required softwares, this is done only once
!pip install -U pip setuptools wheel
!pip install -U spacy
!pip install -U pip setuptools wheel
!python -m spacy download en_core_web_sm


[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
#importing the required modules
import numpy as np
import pandas as pd
import itertools
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [5]:
#we load the data- we are loading fake data
fake_df =pd.read_csv('Fake.csv')
fake_df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [6]:
#we load the data- we are loading real data set.
true_df =pd.read_csv('True.csv')
true_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [8]:
#Checking the size of data
print(fake_df.shape)
print(true_df.shape)

(23481, 4)
(21417, 4)


In [10]:
#We can add a new column called label and name it as true or fake depending on the data and merge them
fake_df['label']='fake'
true_df['label']='true'


                                               title  \
0   Donald Trump Sends Out Embarrassing New Year’...   
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even Has Obama’s Name...   
4   Pope Francis Just Called Out Donald Trump Dur...   

                                                text subject  \
0  Donald Trump just couldn t wish all Americans ...    News   
1  House Intelligence Committee Chairman Devin Nu...    News   
2  On Friday, it was revealed that former Milwauk...    News   
3  On Christmas day, Donald Trump announced that ...    News   
4  Pope Francis used his annual Christmas Day mes...    News   

                date label  
0  December 31, 2017  fake  
1  December 31, 2017  fake  
2  December 30, 2017  fake  
3  December 29, 2017  fake  
4  December 25, 2017  fake  
                                               title  \
0  As U.S. budget fight looms, Republicans flip 

In [11]:
#Checking first 5 rows of fake dataset after adding label column
fake_df.head()


,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",fake
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",fake
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",fake
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",fake
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",fake


In [12]:
#Checking first 5 rows of true dataset after adding label column
true_df.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",true
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",true
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",true
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",true
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",true


In [13]:
#concatnating both the datasets
df=pd.concat([fake_df,true_df], axis=0, ignore_index=True)
df.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",fake
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",fake
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",fake
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",fake
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",fake


In [15]:
#we see that both the datasets are concatenated
df.shape

(44898, 5)

In [16]:
#Splitting data into train and test; our X value is text and y value is label
X_train,X_test,y_train,y_test=train_test_split(df['text'], df['label'], test_size=0.3)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(31428,)
(13470,)
(31428,)
(13470,)


In [17]:
#Using TfidfVectorizer - It tokenizses the documents
tfidf=TfidfVectorizer(stop_words='english', max_df=0.8)
train=tfidf.fit_transform(X_train) 
test=tfidf.transform(X_test)

In [18]:
#Next we apply PassiveAggressiveClassifier: This idea is taken from Geeks for Geeks; 
p=PassiveAggressiveClassifier(max_iter=50)
p.fit(train,y_train)
y_pred=p.predict(test)

In [19]:
#Checking accuracy
s=accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(s*100,2)}%')

Accuracy: 99.41%


In [20]:
#Confusing matrix
confusion_matrix(y_test,y_pred)

array([[7014,   38],
       [  41, 6377]], dtype=int64)

In [22]:
#Classification report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

        fake       0.99      0.99      0.99      7052
        true       0.99      0.99      0.99      6418

    accuracy                           0.99     13470
   macro avg       0.99      0.99      0.99     13470
weighted avg       0.99      0.99      0.99     13470



In [23]:
#now we take the entire dataset not only one column
df['text_new']=df['title']+" "+df['text']+" "+df['subject']
df.head()

,title,text,subject,date,label,text_new
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",fake,Donald Trump Sends Out Embarrassing New Year’...
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",fake,Drunk Bragging Trump Staffer Started Russian ...
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",fake,Sheriff David Clarke Becomes An Internet Joke...
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",fake,Trump Is So Obsessed He Even Has Obama’s Name...
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",fake,Pope Francis Just Called Out Donald Trump Dur...


In [24]:
#applying train , test split on text_new column
X_train,X_test,y_train,y_test=train_test_split(df['text_new'], df['label'], test_size=0.3)

In [25]:
#applying TfidVectorizer
tfidf=TfidfVectorizer(stop_words='english', max_df=0.8)
train=tfidf.fit_transform(X_train) 
test=tfidf.transform(X_test)

In [26]:
#applying passive aggressive classifier
p_1=PassiveAggressiveClassifier(max_iter=50)
p_1.fit(train,y_train)
y_pred=p_1.predict(test)

In [27]:
s1=accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(s1*100,2)}%')

Accuracy: 99.76%


In [28]:
#Confusion matrix
confusion_matrix(y_test,y_pred)

array([[6934,   20],
       [  13, 6503]], dtype=int64)

In [29]:
#classification report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

        fake       1.00      1.00      1.00      6954
        true       1.00      1.00      1.00      6516

    accuracy                           1.00     13470
   macro avg       1.00      1.00      1.00     13470
weighted avg       1.00      1.00      1.00     13470

